In [1]:
%reload_ext autoreload
%autoreload 2

import os
from pathlib import Path
import multiprocessing

import jax
import jax.numpy as jnp
import pandas as pd
import numpy as np
import numpyro
import numpyro.distributions as dist

from hbmep.config import Config
from hbmep.model import Baseline
from hbmep.model.utils import Site as site
from hbmep.utils.constants import RECTIFIED_LOGISTIC

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()


#### Load config

In [2]:
root_path = Path(os.getcwd()).parent.parent.parent.absolute()
toml_path = os.path.join(root_path, "configs/human_midline_lateral.toml")

config = Config(toml_path=toml_path)

config.BUILD_DIR = "/home/vishu/repos/hbmep-paper/reports/human/nb"


2023-07-21 16:28:35,462 - hbmep.config - INFO - Verifying configuration ...
2023-07-21 16:28:35,462 - hbmep.config - INFO - Success!


In [3]:
class RectifiedLogistic(Baseline):
    LINK = RECTIFIED_LOGISTIC

    def __init__(self, config: Config):
        super(RectifiedLogistic, self).__init__(config=config)

    def _model(self, subject, features, intensity, response_obs=None):
        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)

        n_data = intensity.shape[0]
        n_subject = np.unique(subject).shape[0]
        n_feature0 = np.unique(feature0).shape[0]

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate(site.n_subject, n_subject, dim=-2):
                """ Hyper-priors """
                mu_a = numpyro.sample(
                    site.mu_a,
                    dist.TruncatedNormal(3, 10, low=0)
                )
                sigma_a = numpyro.sample(site.sigma_a, dist.HalfNormal(2))

                sigma_b = numpyro.sample(site.sigma_b, dist.HalfNormal(10))

                sigma_L = numpyro.sample(site.sigma_L, dist.HalfNormal(2))
                sigma_H = numpyro.sample(site.sigma_H, dist.HalfNormal(10))
                sigma_v = numpyro.sample(site.sigma_v, dist.HalfNormal(10))

                with numpyro.plate("n_feature0", n_feature0, dim=-3):
                    """ Priors """
                    a = numpyro.sample(
                        site.a,
                        dist.TruncatedNormal(mu_a, sigma_a, low=0)
                    )
                    b = numpyro.sample(site.b, dist.HalfNormal(sigma_b))

                    L = numpyro.sample(site.L, dist.HalfNormal(sigma_L))
                    H = numpyro.sample(site.H, dist.HalfNormal(sigma_H))
                    v = numpyro.sample(site.v, dist.HalfNormal(sigma_v))

                    g_1 = numpyro.sample("g_1", dist.Exponential(1 / 100))
                    g_2 = numpyro.sample("g_2", dist.Exponential(1 / 100))

        """ Model """
        mu = numpyro.deterministic(
            site.mu,
            L[feature0, subject]
            + jnp.maximum(
                0,
                -1
                + (H[feature0, subject] + 1)
                / jnp.power(
                    1
                    + (jnp.power(1 + H[feature0, subject], v[feature0, subject]) - 1)
                    * jnp.exp(-b[feature0, subject] * (intensity - a[feature0, subject])),
                    1 / v[feature0, subject]
                )
            )
        )
        beta = numpyro.deterministic(
            site.beta,
            g_1[feature0, subject] + g_2[feature0, subject] * (1 / mu)
        )

        with numpyro.plate(site.data, n_data):
            return numpyro.sample(
                site.obs,
                dist.Gamma(concentration=mu * beta, rate=beta).to_event(1),
                obs=response_obs
            )


model = RectifiedLogistic(config=config)

In [4]:
df = pd.read_csv(model.csv_path)

df = df.dropna(subset=model.response, axis="rows", how="any").copy()
df.reset_index(drop=True, inplace=True)

ind = df[model.response].apply(lambda x: (x > 0).all(), axis=1)
df = df[ind].copy()
df.reset_index(drop=True, inplace=True)

# sub = ["amap01", "amap02", "amap03", "amap04"]
# sub += ["amap05", "amap06", "amap07", "amap08"]

# ind = df.participant.isin(sub)

# df = df[ind].copy()
# df.reset_index(drop=True, inplace=True)

df, encoder_dict = model.load(df=df)

2023-07-21 16:28:35,774 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/human/nb
2023-07-21 16:28:35,775 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/human/nb
2023-07-21 16:28:35,775 - hbmep.dataset.core - INFO - Processing data ...
2023-07-21 16:28:35,776 - hbmep.utils.utils - INFO - func:load took: 0.00 sec


In [5]:
model.plot(df=df, encoder_dict=encoder_dict)

2023-07-21 16:28:35,925 - hbmep.dataset.core - INFO - Plotting dataset ...


2023-07-21 16:28:39,786 - hbmep.dataset.core - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/human/nb/dataset.pdf
2023-07-21 16:28:39,787 - hbmep.utils.utils - INFO - func:plot took: 3.86 sec


In [6]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-07-21 16:28:39,921 - hbmep.model.baseline - INFO - Running inference with rectified_logistic ...


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

2023-07-21 16:47:01,394 - hbmep.utils.utils - INFO - func:run_inference took: 18 min and 21.47 sec


In [7]:
mcmc.print_summary(prob=.95)



                 mean       std    median      2.5%     97.5%     n_eff     r_hat
   H[0,0,0]      8.65      8.44      5.89      0.34     25.49  10543.95      1.00
   H[0,1,0]      1.49      0.08      1.48      1.35      1.66   5220.02      1.00
   H[0,2,0]      5.87      7.57      3.12      0.01     20.92  10236.38      1.00
   H[0,3,0]      1.13      2.96      0.50      0.38      3.93   1344.73      1.00
   H[0,4,0]      6.72      6.17      3.99      2.86     19.13   4869.31      1.00
   H[0,5,0]      4.47      6.72      1.79      0.00     17.91   3873.58      1.00
   H[0,6,0]      0.80      0.12      0.79      0.59      1.04  12978.22      1.00
   H[0,7,0]     11.55      2.43     11.17      7.73     16.47   5365.07      1.00
   H[0,8,0]     13.59      0.17     13.59     13.27     13.91    462.10      1.00
   H[0,9,0]     12.07      7.60      9.39      4.93     27.65   6842.69      1.00
  H[0,10,0]      0.21      0.02      0.21      0.18      0.25   5641.45      1.00
  H[0,11,0]    

In [8]:
model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-21 16:47:03,119 - hbmep.model.baseline - INFO - Rendering recruitment curves ...


2023-07-21 16:47:41,230 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/human/nb/recruitment_curves.pdf
2023-07-21 16:47:41,230 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 38.11 sec


In [9]:
model.render_predictive_check(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-21 16:47:41,301 - hbmep.model.baseline - INFO - Rendering Posterior Predictive Check ...


2023-07-21 16:48:26,092 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/human/nb/posterior_predictive_check.pdf
2023-07-21 16:48:26,094 - hbmep.utils.utils - INFO - func:render_predictive_check took: 44.79 sec


In [10]:
model.save(mcmc=mcmc)

2023-07-21 16:48:26,188 - hbmep.model.baseline - INFO - Saving inference data ...


2023-07-21 16:48:36,644 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/human/nb/mcmc.nc
2023-07-21 16:48:36,645 - hbmep.model.baseline - INFO - Rendering convergence diagnostics ...
2023-07-21 16:48:58,581 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/human/nb/diagnostics.csv
2023-07-21 16:48:58,581 - hbmep.model.baseline - INFO - Evaluating model ...
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:1037: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This 